<a href="https://colab.research.google.com/github/solankiharsh/learning_deeplearning/blob/main/Pytorch4Use_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Neural Nets using PyTorch

In [1]:
import torch
n_input, n_hidden, n_output = 5, 3, 1

In [2]:
# step 1: Parameter initialization: weights and bias parameters for each layer are initialized as the tensor variables
x = torch.randn((1, n_input))
y = torch.randn((1, n_output))

In [3]:
# initialize tensor variables for weights
w1 = torch.randn(n_input, n_hidden)
w2 = torch.randn(n_hidden, n_output)

In [4]:
# initialize tensor variables for bias terms
b1 = torch.randn((1, n_hidden))
b2 = torch.randn((1, n_output))

After parameter initialization step, a NN can be defined and trained in 4 key steps: 

1. Forward Propagation
2. Loss Computation
3. Back Propagation
4. Updating the parameters

In [5]:
# forward propagation
# z = weight * input + bias
# a = activation_function(z)

In [6]:
def sigmoid_function(z):
  return 1 / (1+torch.exp(-z))

In [7]:
## activation of hidden layer
z1 = torch.mm(x, w1) + b1
a1 = sigmoid_function(z1)

In [8]:
# activation (output) of final layer
z2 = torch.mm(a1, w2) + b2
output = sigmoid_function(z2)

In [9]:
#Loss computation
# the error (called loss) is calculated in the output layer. 
loss = y - output

In [10]:
print(loss)

tensor([[-0.8071]])


# backpropagation
this minimizes the error in the output layer by making marginal changes in the bias and the weights. these marginal changes are computed using the derivatives of the error tem. Based on the calculus principle of the chain rule, the delta changes are back passed to the hidden layers where corresponding changes in their weights and biases are made. 

In [12]:
#fucntion to calculate the derivative of activation
def sigmoid_delta(x):
  return x * (1-x)

In [13]:
# compute derivative of error terms
delta_output = sigmoid_delta(output)
delta_hidden = sigmoid_delta(a1)

In [14]:
# backpass the changes to previous layers
d_outp = loss * delta_output
loss_h = torch.mm(d_outp, w2.t())
d_hidden = loss_h * delta_hidden

In [15]:
#Updating the Parameters: Finally, the weights and bias are updated using the delta changes received from the above backpropagation step.

learning_rate = 0.1

In [18]:
w2 += torch.mm(a1.t(), d_outp) * learning_rate
w1 += torch.mm(x.t(), d_hidden) * learning_rate
b2 += d_outp.sum() * learning_rate
b1 += d_hidden.sum() * learning_rate

# Use Case 1: Handwritten Digital Classification


In [14]:
#!pip install torch==1.8.0
import torch
import torchvision

In [16]:
from torchvision import transforms

In [6]:
_tasks = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5)) #performs normalization (x_norm = x-mean/std) 0.5 represents 
                             #the mean and standard deviation for 3 channels: red, green and blue
])

In [9]:
from torchvision.datasets import MNIST
mnist = MNIST("data", download=True, train=True, transform=_tasks)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


HTTPError: ignored

In [18]:
#dataloader: ability to batch, shuffle and load the data in parallel using multiprocessing workers
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
#creating training and validation split
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
print(index_list)
train_idx, valid_idx = index_list[:split], index_list[split:]

In [ ]:
#create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
## create iterators objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sample=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sample=val_sampler)

In [25]:
#we define the network with following configuration [784,128,10]. 784 nodes (28*28) in the input layer, 128 in hidden, 10 in output
import torch.nn.functional as F
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(784, 128)
    self.output = nn.Linear(128, 10)
  
  def forward(self, x):
    x = self.hidden(x)
    x = F.sigmoid(x)
    x = self.output(x)
    return x 

In [26]:
model = Model()

In [23]:
from torch import optim
import torch
import torch.nn as nn

In [27]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [29]:
for epoch in range(1, 11): ## run the model for 10 epochs
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
                ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = loss_function(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())
        print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

NameError: ignored

In [30]:
# make predictions on the validation data
## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)

NameError: ignored

In [31]:
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

NameError: ignored

In [32]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

NameError: ignored

# Use Case 2: Object Image Classification


In [33]:
## load the dataset 
from torchvision.datasets import CIFAR10
cifar = CIFAR10('data', train=True, download=True, transform=_tasks)

Extracting data/cifar-10-python.tar.gz to data


In [34]:
## create training and validation split 
split = int(0.8 * len(cifar))
index_list = list(range(len(cifar)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [35]:
## create training and validation sampler objects
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [37]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(cifar, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(cifar, batch_size=256, sampler=val_sampler)

In [41]:
# 3 convolutional layer for low-level feature extraction
# 3 pooling layers for maximum information extraction
# 2 linear layers for linear classification
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    #define layers
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.pool = nn.MaxPool2d(2,2)
    self.linear1 = nn.Linear(1024, 512)
    self.linear2 = nn.Linear(512, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 1024) #reshaping
    x = F.relu(self.linear1(x))
    x = self.linear2(x)
    return x

In [42]:
model = Model()

In [43]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [45]:
## run for 30 Epochs
for epoch in range(1, 31):
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item()) 
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())


In [46]:
## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)

In [48]:
import numpy as np
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

In [49]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

Actual: tensor([6, 8, 4, 7, 4, 2, 7, 1, 2, 4])
Predicted: [3 8 0 7 4 6 7 1 2 4]


# Use Case 3: Sentiment Text Classification

In [54]:
import pandas as pd
train = pd.read_csv("train.csv")
train.head()
#x_train = train["text"].values
#y_train = train['label'].values

,text,label
0,A very very very slow-moving aimless movie ...,0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [58]:
x_train = train["text"].values

In [61]:
y_train = train["label"].values

KeyError: ignored

In [62]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [63]:
from keras.preprocessing import text, sequence

In [64]:
## create tokens 
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

NameError: ignored

In [ ]:
## convert texts to padded sequences 
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen = 70)

In [ ]:
!wget https://nlp.stanford.edu/data/wordvecs/glove.840B.300d.zip

In [ ]:
!unzip glove.840B.300d.zip 

In [ ]:
EMBEDDING_FILE = 'glove.840B.300d.txt'

In [ ]:
embeddings_index = {}
for i, line in enumerate(open(EMBEDDING_FILE)):
    val = line.split()
    embeddings_index[val[0]] = np.asarray(val[1:], dtype='float32')

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        ## Embedding Layer, Add parameter 
        self.embedding = nn.Embedding(max_features, embed_size) 
        et = torch.tensor(embedding_matrix, dtype=torch.float32)
        self.embedding.weight = nn.Parameter(et)
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(300, 40)        
        self.linear = nn.Linear(40, 16)
        self.out = nn.Linear(16, 1)
        self.relu = nn.ReLU()
   def forward(self, x):
        h_embedding = self.embedding(x)        
        h_lstm, _ = self.lstm(h_embedding)
        max_pool, _ = torch.max(h_lstm, 1)        
        linear = self.relu(self.linear(max_pool))
        out = self.out(linear)
        return out

In [ ]:
model = Model()

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
## create training and validation split 
split_size = int(0.8 * len(train_df))
index_list = list(range(len(train_df)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [ ]:
## create iterator objects for train and valid datasets
x_tr = torch.tensor(x_train[train_idx], dtype=torch.long)
y_tr = torch.tensor(y_train[train_idx], dtype=torch.float32)
train = TensorDataset(x_tr, y_tr)
trainloader = DataLoader(train, batch_size=128)

In [ ]:
x_val = torch.tensor(x_train[valid_idx], dtype=torch.long)
y_val = torch.tensor(y_train[valid_idx], dtype=torch.float32)
valid = TensorDataset(x_val, y_val)
validloader = DataLoader(valid, batch_size=128)

In [ ]:
loss_function = nn.BCEWithLogitsLoss(reduction='mean')
optimizer = optim.Adam(model.parameters())

In [ ]:
## run for 10 Epochs
for epoch in range(1, 11):
    train_loss, valid_loss = [], []
## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target.view(-1,1))
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target.view(-1,1))
        valid_loss.append(loss.item())

In [ ]:
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

# Use Case #4: Image Style Transfer

In [ ]:
from torchvision import models

In [ ]:
# get the features portion from VGG19
vgg = models.vgg19(pretrainer=True).features
# freeze all VGG parameters
for param in vgg.parameters():
  param.requires_grad_(False)

In [ ]:
#check if GPU is available
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
vgg.to(device)

In [ ]:
from torchvision import transforms as tf

In [ ]:
def transformation(img):
    tasks = tf.Compose([tf.Resize(400), tf.ToTensor(),
               tf.Normalize((0.44,0.44,0.44),(0.22,0.22,0.22))])
    img = tasks(img)[:3,:,:].unsqueeze(0)    
    return img

In [ ]:
img1 = Image.open("image1.jpg").convert('RGB')
img2 = Image.open("image2.jpg").convert('RGB')

In [ ]:
img1 = transformation(img1).to(device)
img2 = transformation(img2).to(device)

Now, we need to obtain the relevant features of the two images. From the first image, we need to extract features related to the context or the objects present. From the second image, we need to extract features related to styles and textures.

Object Related Features: In the original paper, the authors have suggested that more valuable information about objects and context can be extracted from the initial layers of the network. This is because in the higher layers, the information space becomes more complex and detailed pixel information is lost.

Style Related Features: In order to obtain the texture information from the second image, the authors used correlations between different features in different layers. This is explained in detail in point 4 below.



In [ ]:
def get_features(image, model):
    layers = {'0': 'conv1_1', '5': 'conv2_1',  '10': 'conv3_1', 
              '19': 'conv4_1', '21': 'conv4_2', '28': 'conv5_1'}

In [ ]:
x = image
    features = {}
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x     
    return features
img1_features = get_features(img1, vgg)
img2_features = get_features(img2, vgg)

In [ ]:
def correlation_matrix(tensor):
    _, d, h, w = tensor.size()    
    tensor = tensor.view(d, h * w)    
    correlation = torch.mm(tensor, tensor.t())
    return correlation

In [ ]:
correlations = {l: correlation_matrix(img2_features[l]) for l in 
                                                    img2_features}

In [ ]:
weights = {'conv1_1': 1.0, 'conv2_1': 0.8, 'conv3_1': 0.25,
           'conv4_1': 0.21, 'conv5_1': 0.18}

target = img1.clone().requires_grad_(True).to(device)
optimizer = optim.Adam([target], lr=0.003)

In [ ]:
for i in range(1, 2001):
    
    ## calculate the content loss (from image 1 and target)
    target_features = get_features(target, vgg)
    loss = target_features['conv4_2'] - img1_features['conv4_2']
    content_loss = torch.mean((loss)**2)
    
    ## calculate the style loss (from image 2 and target)
    style_loss = 0
    for layer in weights:
        
        target_feature = target_features[layer]
        target_corr = correlation_matrix(target_feature)
        style_corr = correlations[layer]
        
        layer_loss = torch.mean((target_corr - style_corr)**2)
        layer_loss *= weights[layer]
        
        _, d, h, w = target_feature.shape
        style_loss += layer_loss / (d * h * w)   
    
    total_loss = 1e6 * style_loss + content_loss
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [ ]:
def tensor_to_image(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image *= np.array((0.22, 0.22, 0.22)) 
                       + np.array((0.44, 0.44, 0.44))
    image = image.clip(0, 1)
    return image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(tensor_to_image(img1))
ax2.imshow(tensor_to_image(target))